In [ ]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [1]:
% cd /content/gdrive/My\ Drive/Colab\ Notebooks/nlp_task/bert_cla
! pwd
! ls

/content/gdrive/My Drive/Colab Notebooks/nlp_task/bert_cla
/content/gdrive/My Drive/Colab Notebooks/nlp_task/bert_cla
albert_large	  main.py		  run_multi_label_linear.py
albert_tiny	  models		  runs
apex		  outputs		  train.tsv
base_runner.py	  outputs_tiny		  val.tsv
base_utils.py	  __pycache__		  wandb
cache_dir	  run_albert_large.ipynb  xbert_cnn_runner.py
global_config.py  run_albert_tiny.ipynb   xbert_multi_label_linear_runner.py
__init__.py	  run_multi_class_cnn.py  xbert_rnn_runner.py


In [ ]:
!pip install wandb tensorboardX transformers

In [ ]:
# !git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

/content/gdrive/My Drive/Colab Notebooks/nlp_task/bert_cla/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-_6uxp271
Created temporary directory: /tmp/pip-req-tracker-8zf30mlr
Created requirements tracker '/tmp/pip-req-tracker-8zf30mlr'
Created temporary directory: /tmp/pip-install-33er889z
Processing /content/gdrive/My Drive/Colab Notebooks/nlp_task/bert_cla/apex
  Created temporary directory: /tmp/pip-req-build-e6v_9n4s
  Added file:///content/gdrive/My%20Drive/Colab%20Notebooks/nlp_task/bert_cla/apex to build tracker '/tmp/pip-req-tracker-8zf30mlr'
    Running setup.py (path:/tmp/pip-req-build-e6v_9n4s/setup.py) egg_info for package from file:///content/gdrive/My%20Drive/Colab%20Notebooks/nlp_task/bert_cla/apex
    Running command py

In [2]:
!/opt/bin/nvidia-smi

Wed Jul  1 15:08:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from xbert_multi_label_linear_runner import MultiBinaryClaRunner


def process_tsv(file):
    "根据数据的储存方式修改"
    data = pd.read_table(file, names=['label', 'text'], encoding='utf-8')
    label = data.label.apply(lambda x: x.split('@'))
    label = np.array(label.to_list())
    enc = OneHotEncoder()
    # '0_-1', '0_-2', '0_0', '0_1' => 1000, 0100, 0010, 0001
    label_ont_hot = enc.fit_transform(label)
    for i, row_label in enumerate(label_ont_hot.toarray()):
        data.iloc[i]['label'] = row_label
    return data


def main():
    # fine-tuning 阶段 分类器和LM一起训练
    train_df = process_tsv('train.tsv')
    eval_df = process_tsv('val.tsv')
    model = MultiBinaryClaRunner('albert',
                                 './albert_large/',
                                 num_labels=80,
                                 use_cuda=True,
                                 freez_pretrained=False,
                                 args={
                                     "max_seq_length": 512,
                                     "reprocess_input_data": False,
                                     "overwrite_output_dir": False,
                                     "num_train_epochs": 10,
                                     "evaluate_during_training": False,
                                     "use_cached_eval_features": True,
                                 })
    
    model.train_model(train_df)

    result, model_outputs, wrong_predictions = model.eval_model(eval_df)
    print(result)
    print(model_outputs)

    return result, model_outputs, wrong_predictions

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
result, model_outputs, wrong_predictions = main()

INFO:transformers.configuration_utils:loading configuration file ./albert_large/config.json
INFO:transformers.configuration_utils:Model config AlbertConfig {
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "directionality": "bidi",
  "embedding_size": 128,
  "eos_token_id": 3,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27

INFO:base_runner:>>> 开始 fine-tuning.


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.691955

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [ ]:
def main_2():
    # 只训练分类器部分
    train_df = process_tsv('train.tsv')
    eval_df = process_tsv('val.tsv')
    model = MultiBinaryClaRunner('albert',
                                 './outputs/checkpoint-4000',
                                 num_labels=80,
                                 use_cuda=True,
                                 freez_pretrained=True,
                                 args={
                                     "output_dir": './outputs/',
                                     "max_seq_length": 512,
                                     "reprocess_input_data": False,
                                     "overwrite_output_dir": True,
                                     "num_train_epochs": 10,
                                     "evaluate_during_training": True,
                                     "evaluate_during_training_steps": 1000,
                                     "use_cached_eval_features": True,
                                     "save_steps": 1000,
                                 })
    
    model.train_model(train_df, eval_df=eval_df)

    result, model_outputs, wrong_predictions = model.eval_model(eval_df)
    print(result)
    print(model_outputs)

    return result, model_outputs, wrong_predictions

In [ ]:
result, model_outputs, wrong_predictions = main_2()

albert_large训练太慢，前6000步和tiny效果差别不大

In [ ]:
def main_tiny():
    train_df = process_tsv('train.tsv')
    eval_df = process_tsv('val.tsv')
    model = MultiBinaryClaRunner('albert',
                                 './outputs_tiny/checkpoint-16000',
                                 num_labels=80,
                                 use_cuda=True,
                                 freez_pretrained=False,
                                 args={
                                     "train_batch_size": 8,
                                     "output_dir": './outputs_tiny',
                                     "max_seq_length": 512,
                                     "reprocess_input_data": False,
                                     "overwrite_output_dir": True,
                                     "num_train_epochs": 10,
                                     "evaluate_during_training": True,
                                     "evaluate_during_training_steps": 500,
                                     "use_cached_eval_features": True,
                                     "save_steps": 1000,
                                 })
    
    model.train_model(train_df, eval_df=eval_df)

    result, model_outputs, wrong_predictions = model.eval_model(eval_df)
    print(result)
    print(model_outputs)

    return result, model_outputs, wrong_predictions

In [ ]:
result, model_outputs, wrong_predictions = main_tiny()

全局设置： {'sliding_window': False, 'tie_value': -1, 'stride': 0.8, 'regression': False, 'threshold': 0.5, 'best_model_dir': 'outputs/best_model', 'output_dir': './outputs_tiny', 'overwrite_output_dir': True, 'tensorboard_dir': None, 'cache_dir': 'cache_dir/', 'config': {}, 'do_lower_case': False, 'encoding': None, 'train_batch_size': 8, 'num_train_epochs': 10, 'learning_rate': 4e-05, 'adam_epsilon': 1e-06, 'weight_decay': 0, 'warmup_ratio': 0, 'warmup_steps': 1000, 'max_grad_norm': 1.0, 'max_seq_length': 512, 'n_gpu': 1, 'fp16': True, 'fp16_opt_level': 'O1', 'gradient_accumulation_steps': 3, 'eval_batch_size': 8, 'evaluate_during_training': True, 'evaluate_during_training_steps': 500, 'evaluate_during_training_verbose': False, 'use_early_stopping': True, 'early_stopping_consider_epochs': False, 'early_stopping_delta': 0, 'early_stopping_metric': 'eval_loss', 'early_stopping_metric_minimize': True, 'early_stopping_patience': 3, 'manual_seed': 1234, 'save_best_model': True, 'save_eval_check

INFO:transformers.configuration_utils:loading configuration file ./outputs_tiny/checkpoint-16000/config.json
INFO:transformers.configuration_utils:Model config AlbertConfig {
  "architectures": [
    "AlbertForMultiBinaryLabelSeqClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "directionality": "bidi",
  "embedding_size": 128,
  "eos_token_id": 3,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    

模型设置： AlbertConfig {
  "architectures": [
    "AlbertForMultiBinaryLabelSeqClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "directionality": "bidi",
  "embedding_size": 128,
  "eos_token_id": 3,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LABEL_25",
    "26": "LABEL_26",
    "27": "LABEL_27",
    "28": "LABEL_28",
    "29": "LABEL_29",
    

INFO:transformers.modeling_utils:All model checkpoint weights were used when initializing AlbertForMultiBinaryLabelSeqClassification.

INFO:transformers.modeling_utils:All the weights of AlbertForMultiBinaryLabelSeqClassification were initialized from the model checkpoint at ./outputs_tiny/checkpoint-16000.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use AlbertForMultiBinaryLabelSeqClassification for predictions without further training.
INFO:transformers.tokenization_utils_base:Model name './outputs_tiny/checkpoint-16000' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-f

INFO:base_runner:>>> 加载checkpoint，更新global_step.
INFO:base_runner:>>> 已训练 3 轮.
INFO:base_runner:>>> 当前global_step： 16000.
INFO:base_runner:>>> 跳过当前epoch 2875 steps in.


...

Running loss: 0.219279Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2097152.0
Running loss: 0.197920

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.248191

INFO:transformers.configuration_utils:Configuration saved in ./outputs_tiny/checkpoint-30000/config.json
INFO:transformers.modeling_utils:Model weights saved in ./outputs_tiny/checkpoint-30000/pytorch_model.bin
INFO:transformers.configuration_utils:Configuration saved in ./outputs_tiny/checkpoint-30000/config.json
INFO:transformers.modeling_utils:Model weights saved in ./outputs_tiny/checkpoint-30000/pytorch_model.bin
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:base_runner: 达到Early stop最大容忍轮次.
INFO:base_runner: 停止训练@.
INFO:transformers.configuration_utils:Configuration saved in ./outputs_tiny/config.json
INFO:transformers.modeling_utils:Model weights saved in ./outputs_tiny/pytorch_model.bin
INFO:base_runner: 训练 albert 结束. Saved to ./outputs_tiny.
INFO:base_runner: 当前global step： 30000. 运行平局loss： 0.113772


{'LRAP': 0.8801484972131866, 'eval_loss': 0.2407681870142619}
[[0.0026207  0.99121094 0.00252151 ... 0.7211914  0.01634216 0.23742676]
 [0.01908875 0.79003906 0.01428223 ... 0.25561523 0.01634216 0.7109375 ]
 [0.00482941 0.9916992  0.00370789 ... 0.25854492 0.09484863 0.34716797]
 ...
 [0.00666809 0.88427734 0.00793457 ... 0.5576172  0.00462723 0.515625  ]
 [0.00206757 0.9921875  0.00294685 ... 0.76660156 0.01267242 0.26782227]
 [0.02377319 0.03622437 0.03521729 ... 0.17700195 0.01177979 0.7832031 ]]


In [17]:
def main_tiny():
    # fine tuning with fixed albert
    train_df = process_tsv('train.tsv')
    eval_df = process_tsv('val.tsv')
    model = MultiBinaryClaRunner('albert',
                                 './outputs_tiny/checkpoint-32500',
                                 num_labels=80,
                                 use_cuda=True,
                                 freez_pretrained=True,
                                 args={
                                     "train_batch_size": 8,
                                     "learning_rate": 0.0001,
                                     "output_dir": './outputs_tiny',
                                     "best_model_dir": './outputs_tiny/best_model',
                                     "max_seq_length": 512,
                                     "reprocess_input_data": False,
                                     "overwrite_output_dir": True,
                                     "num_train_epochs": 10,
                                     "evaluate_during_training": True,
                                     "evaluate_during_training_steps": 500,
                                     "use_cached_eval_features": True,
                                     "save_steps": 500,
                                 })
    
    model.train_model(train_df, eval_df=eval_df)

    result, model_outputs, wrong_predictions = model.eval_model(eval_df)
    print(result)
    print(model_outputs)

    return result, model_outputs, wrong_predictions

In [18]:
result, model_outputs, wrong_predictions = main_tiny()

INFO:transformers.configuration_utils:loading configuration file ./outputs_tiny/checkpoint-32500/config.json
INFO:transformers.configuration_utils:Model config AlbertConfig {
  "architectures": [
    "AlbertForMultiBinaryLabelSeqClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "directionality": "bidi",
  "embedding_size": 128,
  "eos_token_id": 3,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    

全局设置： {'sliding_window': False, 'tie_value': -1, 'stride': 0.8, 'regression': False, 'threshold': 0.5, 'best_model_dir': './outputs_tiny/best_model', 'output_dir': './outputs_tiny', 'overwrite_output_dir': True, 'tensorboard_dir': None, 'cache_dir': 'cache_dir/', 'config': {}, 'do_lower_case': False, 'encoding': None, 'train_batch_size': 8, 'num_train_epochs': 10, 'learning_rate': 0.0001, 'adam_epsilon': 1e-06, 'weight_decay': 0, 'warmup_ratio': 0, 'warmup_steps': 1000, 'max_grad_norm': 1.0, 'max_seq_length': 512, 'n_gpu': 1, 'fp16': True, 'fp16_opt_level': 'O1', 'gradient_accumulation_steps': 3, 'eval_batch_size': 8, 'evaluate_during_training': True, 'evaluate_during_training_steps': 500, 'evaluate_during_training_verbose': False, 'use_early_stopping': True, 'early_stopping_consider_epochs': False, 'early_stopping_delta': 0, 'early_stopping_metric': 'eval_loss', 'early_stopping_metric_minimize': True, 'early_stopping_patience': 3, 'manual_seed': 1234, 'save_best_model': True, 'save_ev

INFO:transformers.modeling_utils:All model checkpoint weights were used when initializing AlbertForMultiBinaryLabelSeqClassification.

INFO:transformers.modeling_utils:All the weights of AlbertForMultiBinaryLabelSeqClassification were initialized from the model checkpoint at ./outputs_tiny/checkpoint-32500.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use AlbertForMultiBinaryLabelSeqClassification for predictions without further training.
INFO:transformers.tokenization_utils_base:Model name './outputs_tiny/checkpoint-32500' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-f

INFO:base_runner:>>> 加载checkpoint，更新global_step.
INFO:base_runner:>>> 已训练 7 轮.
INFO:base_runner:>>> 当前global_step： 32500.
INFO:base_runner:>>> 跳过当前epoch 1875 steps in.


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


...

Running loss: 0.247618

INFO:transformers.configuration_utils:Configuration saved in ./outputs_tiny/checkpoint-45000/config.json
INFO:transformers.modeling_utils:Model weights saved in ./outputs_tiny/checkpoint-45000/pytorch_model.bin
INFO:transformers.configuration_utils:Configuration saved in ./outputs_tiny/checkpoint-45000/config.json
INFO:transformers.modeling_utils:Model weights saved in ./outputs_tiny/checkpoint-45000/pytorch_model.bin
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:transformers.configuration_utils:Configuration saved in ./outputs_tiny/best_model/config.json
INFO:transformers.modeling_utils:Model weights saved in ./outputs_tiny/best_model/pytorch_model.bin
INFO:transformers.configuration_utils:Configuration saved in ./outputs_tiny/checkpoint-45000-epoch-3/config.json
INFO:transformers.modeling_utils:Model weig

INFO:transformers.configuration_utils:Configuration saved in ./outputs_tiny/config.json
INFO:transformers.modeling_utils:Model weights saved in ./outputs_tiny/pytorch_model.bin


INFO:base_runner: 训练 albert 结束. Saved to ./outputs_tiny.
INFO:base_runner: 当前global step： 45000. 运行平局loss： 0.06435975685798459.



{'LRAP': 0.8821297770655848, 'eval_loss': 0.23851567214330038}
[[0.00300407 0.9946289  0.00234985 ... 0.77001953 0.02546692 0.15515137]
 [0.01652527 0.8105469  0.01040649 ... 0.22033691 0.01869202 0.71875   ]
 [0.00344276 0.9970703  0.00202751 ... 0.27783203 0.13757324 0.23400879]
 ...
 [0.00542831 0.9482422  0.00593567 ... 0.3935547  0.00566483 0.6254883 ]
 [0.0017004  0.9951172  0.00203514 ... 0.6191406  0.01428223 0.3701172 ]
 [0.0236969  0.01864624 0.0292511  ... 0.05706787 0.01205444 0.9223633 ]]


## 测试

In [5]:
eval_df = process_tsv('val.tsv')
model = MultiBinaryClaRunner('albert',
                                './outputs/best_model',
                                num_labels=80,
                                use_cuda=True,
                                freez_pretrained=False,
                                args={
                                    "output_dir": './outputs_tiny',
                                    "max_seq_length": 512,
                                    "reprocess_input_data": False,
                                    "overwrite_output_dir": True,
                                    "use_cached_eval_features": True,
                                })

result, model_outputs, wrong_predictions = model.eval_model(eval_df)

INFO:transformers.configuration_utils:loading configuration file ./outputs/best_model/config.json
INFO:transformers.configuration_utils:Model config AlbertConfig {
  "architectures": [
    "AlbertForMultiBinaryLabelSeqClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "directionality": "bidi",
  "embedding_size": 128,
  "eos_token_id": 3,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABE

全局设置： {'sliding_window': False, 'tie_value': -1, 'stride': 0.8, 'regression': False, 'threshold': 0.5, 'best_model_dir': 'outputs/best_model', 'output_dir': './outputs_tiny', 'overwrite_output_dir': True, 'tensorboard_dir': None, 'cache_dir': 'cache_dir/', 'config': {}, 'do_lower_case': False, 'encoding': None, 'train_batch_size': 8, 'num_train_epochs': 10, 'learning_rate': 4e-05, 'adam_epsilon': 1e-06, 'weight_decay': 0, 'warmup_ratio': 0, 'warmup_steps': 1000, 'max_grad_norm': 1.0, 'max_seq_length': 512, 'n_gpu': 1, 'fp16': True, 'fp16_opt_level': 'O1', 'gradient_accumulation_steps': 3, 'eval_batch_size': 8, 'evaluate_during_training': True, 'evaluate_during_training_steps': 500, 'evaluate_during_training_verbose': False, 'use_early_stopping': True, 'early_stopping_consider_epochs': False, 'early_stopping_delta': 0, 'early_stopping_metric': 'eval_loss', 'early_stopping_metric_minimize': True, 'early_stopping_patience': 3, 'manual_seed': 1234, 'save_best_model': True, 'save_eval_check

INFO:transformers.modeling_utils:All model checkpoint weights were used when initializing AlbertForMultiBinaryLabelSeqClassification.

INFO:transformers.modeling_utils:All the weights of AlbertForMultiBinaryLabelSeqClassification were initialized from the model checkpoint at ./outputs/best_model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use AlbertForMultiBinaryLabelSeqClassification for predictions without further training.
INFO:transformers.tokenization_utils_base:Model name './outputs/best_model' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-ba

Freeze参数： False


In [20]:
model_outputs.shape

(15000, 80)

In [21]:
m = np.hsplit(model_outputs, 20)  # 每种label的预测
print(m[0].shape)

(15000, 4)


In [22]:
y_pre = []
for lab in m:
    lab_pre = []
    for item in lab:
        i = np.argmax(item)
        new = np.zeros((4))
        new[i] = 1
        lab_pre.append(new)
    y_pre.append(lab_pre)

print(len(y_pre))
print(len(y_pre[0]))

20
15000


In [23]:
y = eval_df.label.to_numpy()
y_split = [np.split(ar, 20) for ar in y]
y_zip = list(zip(*y_split))

print(y_split[0])

[array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 0., 1., 0.]), array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 0., 0., 1.]), array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 0., 1., 0.]), array([0., 1., 0., 0.]), array([0., 1., 0., 0.]), array([0., 0., 0., 1.]), array([0., 0., 1., 0.])]


In [24]:
def cal_f1(label_num,predicted,truth):
    results = []
    for i in range(label_num):
        results.append({"TP": 0, "FP": 0, "FN": 0, "TN": 0})
    
    for i, p in enumerate(predicted):
        t = truth[i]
        for j in range(label_num):
            if p[j] == 1:
                if t[j] == 1:
                    results[j]['TP'] += 1
                else:
                    results[j]['FP'] += 1
            else:
                if t[j] == 1:
                    results[j]['FN'] += 1
                else:
                    results[j]['TN'] += 1
    
    precision = [0.0] * label_num
    recall = [0.0] * label_num
    f1 = [0.0] * label_num
    for i in range(label_num):
        if results[i]['TP'] == 0:
            if results[i]['FP']==0 and results[i]['FN']==0:
                precision[i] = 1.0
                recall[i] = 1.0
                f1[i] = 1.0
            else:
                precision[i] = 0.0
                recall[i] = 0.0
                f1[i] = 0.0
        else:
            precision[i] = results[i]['TP'] / (results[i]['TP'] + results[i]['FP'])
            recall[i] = results[i]['TP'] / (results[i]['TP'] + results[i]['FN'])
            f1[i] =  2 * precision[i] * recall[i] / (precision[i] + recall[i])
    
    # for i in range(label_num):
    #     print(i,results[i], precision[i], recall[i], f1[i])
    return sum(f1)/label_num, sum(precision)/label_num, sum(recall)/label_num

In [25]:
results = {}
total_f1 = 0
for i in range(20):
    # print("# Get f1 score for",label_name)
    f1,precision,recall = cal_f1(4, y_pre[i], y_zip[i])
    results[i] = f1
    total_f1 += f1
    print("# {0} - {1}".format(i,f1))

final_f1 = total_f1 / len(results)
    
print(final_f1)

# 0 - 0.44064020977293755
# 1 - 0.3696688644733156
# 2 - 0.3982878840786615
# 3 - 0.34883744707704983
# 4 - 0.6016700385214818
# 5 - 0.45781193386366087
# 6 - 0.42781576926606496
# 7 - 0.524714241799282
# 8 - 0.4495948166820826
# 9 - 0.5343606252608031
# 10 - 0.4363348338622312
# 11 - 0.5021951116568629
# 12 - 0.3748723510664493
# 13 - 0.49197442669459646
# 14 - 0.4101902428777168
# 15 - 0.506711133024883
# 16 - 0.3371772537035236
# 17 - 0.3672668029606441
# 18 - 0.5164810910196235
# 19 - 0.5158296444530948
0.4506217361057483
